### Gisement d'or

**Impact de la teneur de coupure, de la corrélation spatiale sur la localiation des ressources**

Tout au long de la session, nous allons apprendre à choisir une teneur de coupure, à estimer la corrélation spatiale d’un gisement, et surtout à comprendre comment ces paramètres influencent l’estimation des ressources.

Mais pour l’instant… amusez-vous un peu !

Explorez ce modèle 3D interactif et observez comment la localisation des ressources évolue lorsque vous modifiez la teneur de coupure ou le degré de corrélation spatiale.
Un bon moyen de jouer les géologues tout en développant votre intuition !

**Cette image interactive est encore en développement.**


In [32]:
import numpy as np
from scipy.fftpack import fftn, ifftn
from ipywidgets import interact, FloatSlider, IntSlider
import plotly.graph_objects as go
import plotly.io as pio

def spherical_covariance_fft_3d(shape, ax, ay, az, angle_x, angle_y, angle_z, sill=1.0):
    nx, ny, nz = shape
    x = np.arange(-nx, nx)
    y = np.arange(-ny, ny)
    z = np.arange(-nz, nz)
    X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

    X_scaled = X / ax
    Y_scaled = Y / ay
    Z_scaled = Z / az

    def rotation_matrix(rx, ry, rz):
        rx = np.radians(rx)
        ry = np.radians(ry)
        rz = np.radians(rz)
        Rx = np.array([[1, 0, 0],
                       [0, np.cos(rx), -np.sin(rx)],
                       [0, np.sin(rx), np.cos(rx)]])
        Ry = np.array([[np.cos(ry), 0, np.sin(ry)],
                       [0, 1, 0],
                       [-np.sin(ry), 0, np.cos(ry)]])
        Rz = np.array([[np.cos(rz), -np.sin(rz), 0],
                       [np.sin(rz), np.cos(rz), 0],
                       [0, 0, 1]])
        return Rz @ Ry @ Rx

    R = rotation_matrix(angle_x, angle_y, angle_z)
    coords = np.stack([X_scaled, Y_scaled, Z_scaled], axis=-1)
    coords_rot = coords @ R.T

    h = np.linalg.norm(coords_rot, axis=-1)
    h = np.minimum(h, 1.0)
    cov = sill * (1 - 1.5 * h + 0.5 * h**3)
    cov[h > 1] = 0
    return cov

def fftma_3d(shape, ax, ay, az, angle_x, angle_y, angle_z, sill=1.0, seed=0):
    np.random.seed(seed)
    cov = spherical_covariance_fft_3d(shape, ax, ay, az, angle_x, angle_y, angle_z, sill)
    white_noise = np.random.normal(size=cov.shape)
    cov_fft = fftn(cov)
    white_fft = fftn(white_noise)
    z_fft = np.sqrt(np.abs(cov_fft)) * white_fft
    field = np.real(ifftn(z_fft))
    start = np.array(shape)
    end = start * 2
    slices = (slice(start[0], end[0]), slice(start[1], end[1]), slice(start[2], end[2]))
    return field[slices]

def gaussian_to_lognormal(field, mean, variance):
    sigma = np.sqrt(np.log(variance / mean**2 + 1))
    mu = np.log(mean) - 0.5 * sigma**2
    return np.exp(field * sigma + mu)

def plot_plotly_3d(field, cutoff):
    nx, ny, nz = field.shape
    spacing = (1, 1, 1)

    mask = field >= cutoff
    coords = np.argwhere(mask)

    if len(coords) == 0:
        print("Aucun voxel au-dessus du seuil cutoff.")
        return

    x = coords[:, 0] * spacing[0]
    y = coords[:, 1] * spacing[1]
    z = coords[:, 2] * spacing[2]

    values = field[mask]

    fig = go.Figure(data=go.Scatter3d(
    x=x, y=y, z=z,
    mode='markers',
    marker=dict(
        size=5,
        color=values,
        colorscale='Viridis',
        cmin=0,      # <-- Ajout ici
        cmax=10,     # <-- Ajout ici
        opacity=0.8,
        colorbar=dict(title='Teneur (ppm)'),
        showscale=True,
        symbol='square'
    )
    ))

    fig.update_layout(
        scene=dict(
            xaxis_title='X (m)',
            yaxis_title='Y (m)',
            zaxis_title='Z (m)',
            aspectratio=dict(x=nx*spacing[0], y=ny*spacing[1], z=nz*spacing[2]),
            camera=dict(
                eye=dict(x=20000, y=20000, z=20000) 
            )
        ),
        title='Simulation 3D d\'un gisement d\'or'
    )
    fig.show()


def interactive_sim_3d(mean, variance, cutoff, ax, ay, az, angle_x, angle_y, angle_z):
    shape = (100, 100, 50)
    field = fftma_3d(shape, ax, ay, az, angle_x, angle_y, angle_z, sill=variance)
    field = gaussian_to_lognormal(field, mean, variance)
    plot_plotly_3d(field, cutoff)

interact(interactive_sim_3d,
         mean=FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='Moyenne'),
         variance=FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Variance'),
         cutoff=FloatSlider(value=1.0, min=0.0, max=10.0, step=0.1, description='Teneur coupure'),
         ax=FloatSlider(value=10.0, min=1.0, max=50.0, step=1.0, description='Portée Ax'),
         ay=FloatSlider(value=10.0, min=1.0, max=50.0, step=1.0, description='Portée Ay'),
         az=FloatSlider(value=10.0, min=1.0, max=50.0, step=1.0, description='Portée Az'),
         angle_x=IntSlider(value=0, min=0, max=360, step=1, description='Angle X'),
         angle_y=IntSlider(value=0, min=0, max=360, step=1, description='Angle Y'),
         angle_z=IntSlider(value=0, min=0, max=360, step=1, description='Angle Z'))


interactive(children=(FloatSlider(value=1.0, description='Moyenne', max=10.0, min=0.1), FloatSlider(value=1.0,…

<function __main__.interactive_sim_3d(mean, variance, cutoff, ax, ay, az, angle_x, angle_y, angle_z)>